In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

rng = np.random.RandomState(1234)
random_state = 42

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
mnist_X, mnist_y = mnist.train.images, mnist.train.labels
mnist_X = mnist_X.reshape((mnist_X.shape[0], 28, 28, 1))

train_X, valid_X, train_y, valid_y = train_test_split(mnist_X, mnist_y, test_size=0.1, random_state=42)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# 入力 (4次元)
x = tf.placeholder(tf.float32)

# サンプル画像
sample_image = np.array([[1, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 1, 1, 1],
                         [0, 0, 1, 1, 0],
                         [0, 1, 1, 0, 0]]
                       ).astype('float32').reshape(1, 5, 5, 1)

# フィルタ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(3, 3, 1, 1)

In [6]:
convoluted_image = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')

with tf.Session() as sess:
    res = sess.run(convoluted_image, feed_dict={x: sample_image})
    print(res.shape, res)
#     print(sess.run(convoluted_image, feed_dict={x: sample_image}).reshape(3, 3))

(1, 3, 3, 1) [[[[ 4.]
   [ 3.]
   [ 4.]]

  [[ 2.]
   [ 4.]
   [ 3.]]

  [[ 2.]
   [ 3.]
   [ 4.]]]]


In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in range(100):
    ax = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(weight_1[i].reshape((28, 28)), cmap='gray')

In [7]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import tensorflow as tf

del [
    tf.app,
    tf.compat,
    tf.contrib,
    tf.errors,
    tf.gfile,
    tf.graph_util,
    tf.image,
    tf.layers,
    tf.logging,
    tf.losses,
    tf.metrics,
    tf.python_io,
    tf.resource_loader,
    tf.saved_model,
    tf.sdca,
    tf.sets,
    tf.summary,
    tf.sysconfig,
    tf.test
]

def load_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
    mnist_X = np.r_[mnist.train.images, mnist.test.images]
    mnist_y = np.r_[mnist.train.labels, mnist.test.labels]
    return train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))

    # validate for small dataset
    train_X_mini = train_X[:100]
    train_y_mini = train_y[:100]
    test_X_mini = test_X[:100]
    test_y_mini = test_y[:100]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(np.argmax(test_y_mini, 1), pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))
    
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

In [10]:
def homework(train_X, train_y, test_X):
    t_X, v_X, t_y, v_y = train_test_split(train_X, train_y, test_size=0.1, random_state=42)
    
    tf.reset_default_graph()
    
    X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
    t = tf.placeholder(tf.float32, [None, 10], name='t')
    
    with tf.variable_scope('filters'):
        f1 = tf.Variable(tf.truncated_normal([10, 10, 1, 5]), name='f1')
        b1 = tf.Variable(tf.zeros(5), name='b1')
        f2 = tf.Variable(tf.truncated_normal([5, 5, 5, 8]), name='f2')
        b2 = tf.Variable(tf.zeros(8), name='b2')

        f3 = tf.Variable(tf.truncated_normal([10, 10, 8, 8]), name='f1')
        b3 = tf.Variable(tf.zeros(8), name='b1')
        f4= tf.Variable(tf.truncated_normal([5, 5, 8, 8]), name='f2')
        b4 = tf.Variable(tf.zeros(8), name='b2')

        W1 = tf.Variable(tf.truncated_normal([28 * 28 * 8, 10]), name='W1')
        b5 = tf.Variable(tf.zeros(10), name='b5')
        
    # Convolution 1
    u1 = tf.nn.conv2d(X, f1, strides=[1,1,1,1], padding='SAME') + b1
    z1 = tf.nn.relu(u1)
    # Convolution 2
    u2 = tf.nn.conv2d(z1, f2, strides=[1,1,1,1], padding='SAME') + b2
    z2 = tf.nn.relu(u2)
    
    # Max Pooling
    p1 = tf.nn.max_pool(z2, ksize=[1, 4, 4, 1], strides=[1,1,1,1], padding='SAME')
    
    # Convolution 3
    u3 = tf.nn.conv2d(p1, f3, strides=[1,1,1,1], padding='SAME') + b3
    z3 = tf.nn.relu(u3)
    # Convolution 4
    u4 = tf.nn.conv2d(z3, f3, strides=[1,1,1,1], padding='SAME') + b4
    z4 = tf.nn.relu(u4)
    
    # Max Pooling
    p2 = tf.nn.max_pool(z4, ksize=[1, 4, 4, 1], strides=[1,1,1,1], padding='SAME')
    
    # FC layer
#     print(p2)
    flattened = tf.reshape(p2, [-1, np.prod(p2.shape.as_list()[1:])])
    u5 = tf.matmul(flattened, W1) + b5
    predicted = tf.argmax(u5, axis=1)
    
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=u5, labels=t)
    train = tf.train.AdamOptimizer().minimize(cost)
    
    # Train
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        epochs = 10
        batch_size = 10
        steps_in_epoch = t_X.shape[0] // batch_size
        
        for epoch in range(epochs):
            for step in range(steps_in_epoch):
                start_idx = step * batch_size
                end_idx = (step + 1) * batch_size
                _, c = sess.run([train, cost],
                                           feed_dict={X: t_X[start_idx:end_idx], t: t_y[start_idx:end_idx]})
            c, p = sess.run(cost, predicted, feed_dict={X: v_X, t: v_y})
            format_str = "Epoch: %d   ---Valid cost %f   ---Valid f %f"
            print(format_str % (epoch, c, f_score(np.argmax(v_y, 1).astype('int32'),
                                                                               p, average='macro')))
        pred_y = sess.run(predicted, feed_dict={X: test_X})
        
    return pred_y

In [ ]:
def homework(train_X, train_y, test_X):
    t_X, v_X, t_y, v_y = train_test_split(train_X, train_y, test_size=0.1, random_state=42)
    
    tf.reset_default_graph()
    
    X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
    t = tf.placeholder(tf.float32, [None, 10], name='t')
    
    with tf.variable_scope('filters'):
        f1 = tf.Variable(tf.truncated_normal([5, 5, 1, 1]), name='f1')
        b1 = tf.Variable(tf.zeros(1), name='b1')
        f2 = tf.Variable(tf.truncated_normal([5, 5, 1, 1]), name='f2')
        b2 = tf.Variable(tf.zeros(1), name='b2')

        f3 = tf.Variable(tf.truncated_normal([5, 5, 1, 1]), name='f1')
        b3 = tf.Variable(tf.zeros(1), name='b1')
#         f4= tf.Variable(tf.truncated_normal([5, 5, 1, 1]), name='f2')
#         b4 = tf.Variable(tf.zeros(1), name='b2')

        W1 = tf.Variable(tf.truncated_normal([28 * 28 * 1, 10]), name='W1')
        b5 = tf.Variable(tf.zeros(10), name='b5')
        
    # Convolution 1
    u1 = tf.nn.conv2d(X, f1, strides=[1,1,1,1], padding='SAME') + b1
    z1 = tf.nn.relu(u1)
    # Convolution 2
    u2 = tf.nn.conv2d(z1, f2, strides=[1,1,1,1], padding='SAME') + b2
    z2 = tf.nn.relu(u2)
    
    # Max Pooling
    p1 = tf.nn.max_pool(z2, ksize=[1, 4, 4, 1], strides=[1,1,1,1], padding='SAME')
    
    # Convolution 3
    u3 = tf.nn.conv2d(p1, f3, strides=[1,1,1,1], padding='SAME') + b3
    z3 = tf.nn.relu(u3)
    # Convolution 4
#     u4 = tf.nn.conv2d(z3, f3, strides=[1,1,1,1], padding='SAME') + b4
#     z4 = tf.nn.relu(u4)
    
    # Max Pooling
    p2 = tf.nn.max_pool(z3, ksize=[1, 8, 8, 1], strides=[1,1,1,1], padding='SAME')
    
    # FC layer
    flattened = tf.reshape(p2, [-1, np.prod(p2.shape.as_list()[1:])])
    u5 = tf.matmul(flattened, W1) + b5
    predicted = tf.argmax(u5, axis=1)
    
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=u5, labels=t)
    train = tf.train.AdamOptimizer().minimize(cost)
    
    # Train
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        epochs = 10
        batch_size = 10
        steps_in_epoch = t_X.shape[0] // batch_size
        
        for epoch in range(epochs):
            for step in range(steps_in_epoch):
                start_idx = step * batch_size
                end_idx = (step + 1) * batch_size
                _, c = sess.run([train, cost],
                                           feed_dict={X: t_X[start_idx:end_idx], t: t_y[start_idx:end_idx]})
            c, p = sess.run(cost, predicted, feed_dict={X: v_X, t: v_y})
            format_str = "Epoch: %d   ---Valid cost %f   ---Valid f %f"
            print(format_str % (epoch, c, f_score(np.argmax(v_y, 1).astype('int32'),
                                                                               p, average='macro')))
        pred_y = sess.run(predicted, feed_dict={X: test_X})
        
    return pred_y